In [ ]:
import requests

from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM 


model = AutoModelForCausalLM.from_pretrained("microsoft/Florence-2-large-ft", trust_remote_code=True, device_map="cuda").eval()
processor = AutoProcessor.from_pretrained("microsoft/Florence-2-large-ft", trust_remote_code=True)

# **OCR Benchmark**
***

In [ ]:
import os
from tqdm import tqdm
from glob import glob

test_set_dir = '/mnt/opr/levlevi/player-re-id/src/testing/ocr_model_comparions/nba_100_test_set'
image_file_paths = glob(os.path.join(test_set_dir, '*.jpg'))
PROMPT = """Analyze the basketball player shown in the provided still tracklet frame and describe the following details:
1. Jersey Number: Identify the number on the player's jersey. If the player has no jersey, provide None.
Based on the frame description, produce an output prediction in the following JSON format:
{
  "jersey_number": "<predicted_jersey_number>",
}
[EOS]"""

def perform_ocr(image_file_path: str) -> str:
    image = Image.open(image_file_path)
    inputs = processor(text=PROMPT, images=image, return_tensors="pt").to('cuda')
    generated_ids = model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        max_new_tokens=1024,
        do_sample=True,
        num_beams=10
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(generated_text, task="<OCR>", image_size=(image.width, image.height))
    return parsed_answer

ground_truth_labels = []
results = []
for image_file_path in tqdm(image_file_paths):
    # get human annotation
    ground_truth_label = image_file_path.split('/')[-1].split('_')[1].split('.')[0]
    ground_truth_labels.append(ground_truth_label)
    # perform ocr
    result = perform_ocr(image_file_path)
    results.append(result)

In [ ]:
import pandas as pd

ground_truth_labels = [f.split('/')[-1].split('_')[1].split('.')[0] for f in image_file_paths]
predictions = []
for r in results:
    pred = r['<OCR>']
    if pred == 'unanswerable':
        pred = 'nan'
    predictions.append(pred)
    
df = pd.DataFrame({'ground_truth_label': ground_truth_labels, 'prediction': predictions})
correct = df['ground_truth_label'] == df['prediction']
df['correct'] = correct
sum(df['correct'])

# **Race Identification Benchmark**
***

In [ ]:
import os
from tqdm import tqdm
from glob import glob

test_set_dir = '/mnt/opr/levlevi/player-re-id/src/testing/ocr_model_comparions/nba_100_test_set'
image_file_paths = glob(os.path.join(test_set_dir, '*.jpg'))
PROMPT = """Analyze the basketball player shown in the provided still tracklet frame and describe the following details:
1. Player Race: Identify the players skin color as either 'black', 'white', 'mixed', or 'asian'.
Based on the frame description, produce an output prediction in the following JSON format:
{
  "race": "<predicted_race>",
}
[EOS]"""

def perform_ocr(image_file_path: str) -> str:
    image = Image.open(image_file_path)
    inputs = processor(text=PROMPT, images=image, return_tensors="pt").to('cuda')
    generated_ids = model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        max_new_tokens=1024,
        do_sample=False,
        num_beams=3
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(generated_text, task="<CAPTION_TO_PHRASE_GROUNDING>", image_size=(image.width, image.height))
    return parsed_answer

ground_truth_labels = []
results = []
for image_file_path in tqdm(image_file_paths):
    # get human annotation
    ground_truth_label = image_file_path.split('/')[-1].split('_')[1].split('.')[0]
    ground_truth_labels.append(ground_truth_label)
    # perform ocr
    result = perform_ocr(image_file_path)
    results.append(result)